# Importing the libraries

In [1]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

In [2]:
orders=pd.read_csv("Data/order_products__prior.csv")

In [3]:
orders.columns

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')

In [4]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB


In [5]:
orders.isna().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

In [6]:
orders = orders.set_index('order_id')['product_id'].rename('item_id')
orders

order_id
2          33120
2          28985
2           9327
2          45918
2          30035
           ...  
3421083    39678
3421083    11352
3421083     4600
3421083    24852
3421083     5020
Name: item_id, Length: 32434489, dtype: int64

In [7]:
orders

order_id
2          33120
2          28985
2           9327
2          45918
2          30035
           ...  
3421083    39678
3421083    11352
3421083     4600
3421083    24852
3421083     5020
Name: item_id, Length: 32434489, dtype: int64

In [8]:
df = orders.groupby('order_id').apply(list).to_dict()

In [9]:
transactions = []
for i in df.values():
    transactions.append(i)

In [10]:
len(transactions)

3214874

In [11]:
len(orders.index.unique())

3214874

# Performing the Apriori Association Rule Using Apyori Library

In [12]:
from apyori  import apriori
rules = apriori(transactions = transactions, min_support=0.001,min_confidence = 0.2,min_lift=3,min_length=2,max_length=2)

In [13]:
results =  list(rules)
len(results)

64

In [14]:
results

[RelationRecord(items=frozenset({196, 46149}), support=0.0012348850996959757, ordered_statistics=[OrderedStatistic(items_base=frozenset({46149}), items_add=frozenset({196}), confidence=0.46389343304510405, lift=41.668546189473496)]),
 RelationRecord(items=frozenset({1158, 21903}), support=0.0012675457887307558, ordered_statistics=[OrderedStatistic(items_base=frozenset({1158}), items_add=frozenset({21903}), confidence=0.2302389965534776, lift=3.0596325403989932)]),
 RelationRecord(items=frozenset({15290, 2295}), support=0.0022159499874645164, ordered_statistics=[OrderedStatistic(items_base=frozenset({2295}), items_add=frozenset({15290}), confidence=0.2798224596409914, lift=22.955852559862013)]),
 RelationRecord(items=frozenset({33754, 4957}), support=0.002902446565557468, ordered_statistics=[OrderedStatistic(items_base=frozenset({4957}), items_add=frozenset({33754}), confidence=0.449513440601214, lift=48.343393866102005), OrderedStatistic(items_base=frozenset({33754}), items_add=frozens

In [15]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))

In [16]:
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

In [17]:
resultsinDataFrame

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,46149,196,0.001235,0.463893,41.668546
1,1158,21903,0.001268,0.230239,3.059633
2,2295,15290,0.002216,0.279822,22.955853
3,4957,33754,0.002902,0.449513,48.343394
4,4957,33787,0.001954,0.302630,48.870721
...,...,...,...,...,...
59,36316,47977,0.001037,0.351201,75.634068
60,36695,47209,0.001495,0.201155,3.027795
61,41065,45007,0.001553,0.444801,13.641841
62,41950,47209,0.004063,0.203161,3.057983


In [18]:
products = pd.read_csv("Data/products.csv")

In [19]:
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [20]:
output1 = pd.merge(resultsinDataFrame,products,left_on='Left Hand Side',right_on='product_id')
output1

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,product_id,product_name,aisle_id,department_id
0,46149,196,0.001235,0.463893,41.668546,46149,Zero Calorie Cola,77,7
1,1158,21903,0.001268,0.230239,3.059633,1158,Mango Chunks,116,1
2,2295,15290,0.002216,0.279822,22.955853,2295,Yellow Bell Pepper,83,4
3,4957,33754,0.002902,0.449513,48.343394,4957,Total 2% Lowfat Greek Strained Yogurt With Blu...,120,16
4,4957,33787,0.001954,0.302630,48.870721,4957,Total 2% Lowfat Greek Strained Yogurt With Blu...,120,16
...,...,...,...,...,...,...,...,...,...
59,36316,47977,0.001037,0.351201,75.634068,36316,Lemon Sparkling Water,115,7
60,36695,47209,0.001495,0.201155,3.027795,36695,Organic Hothouse Cucumbers,83,4
61,41065,45007,0.001553,0.444801,13.641841,41065,Organic Yellow Squash,83,4
62,41950,47209,0.004063,0.203161,3.057983,41950,Organic Tomato Cluster,83,4


In [21]:
output =  pd.merge(output1,products,left_on='Right Hand Side',right_on='product_id')
output

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,product_id_x,product_name_x,aisle_id_x,department_id_x,product_id_y,product_name_y,aisle_id_y,department_id_y
0,46149,196,0.001235,0.463893,41.668546,46149,Zero Calorie Cola,77,7,196,Soda,77,7
1,1158,21903,0.001268,0.230239,3.059633,1158,Mango Chunks,116,1,21903,Organic Baby Spinach,123,4
2,33120,21903,0.001391,0.230464,3.062622,33120,Organic Egg Whites,86,16,21903,Organic Baby Spinach,123,4
3,40545,21903,0.001008,0.227806,3.027304,40545,Berry Medley,116,1,21903,Organic Baby Spinach,123,4
4,40604,21903,0.002281,0.226701,3.012615,40604,Feta Cheese Crumbles,21,16,21903,Organic Baby Spinach,123,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,35939,31717,0.001308,0.207482,9.594218,35939,Organic Jalapeno Pepper,83,4,31717,Organic Cilantro,16,4
60,39984,34126,0.001006,0.202770,10.753394,39984,Organic Dill,16,4,34126,Organic Italian Parsley Bunch,16,4
61,36316,47977,0.001037,0.351201,75.634068,36316,Lemon Sparkling Water,115,7,47977,Grapefruit Sparkling Water,115,7
62,41065,45007,0.001553,0.444801,13.641841,41065,Organic Yellow Squash,83,4,45007,Organic Zucchini,83,4


In [22]:
resultsdf = output[['product_name_x','product_name_y','Lift']]
resultsdf

,product_name_x,product_name_y,Lift
0,Zero Calorie Cola,Soda,41.668546
1,Mango Chunks,Organic Baby Spinach,3.059633
2,Organic Egg Whites,Organic Baby Spinach,3.062622
3,Berry Medley,Organic Baby Spinach,3.027304
4,Feta Cheese Crumbles,Organic Baby Spinach,3.012615
...,...,...,...
59,Organic Jalapeno Pepper,Organic Cilantro,9.594218
60,Organic Dill,Organic Italian Parsley Bunch,10.753394
61,Lemon Sparkling Water,Grapefruit Sparkling Water,75.634068
62,Organic Yellow Squash,Organic Zucchini,13.641841


In [89]:
resultsdf.nlargest(n=10,columns='Lift')

,product_name_x,product_name_y,Lift
24,Non Fat Acai & Mixed Berries Yogurt,Icelandic Style Skyr Blueberry Non-fat Yogurt,75.647888
61,Lemon Sparkling Water,Grapefruit Sparkling Water,75.634068
28,Icelandic Style Skyr Blueberry Non-fat Yogurt,Non Fat Raspberry Yogurt,73.640837
23,Nonfat Icelandic Style Strawberry Yogurt,Icelandic Style Skyr Blueberry Non-fat Yogurt,71.378186
26,Nonfat Icelandic Style Strawberry Yogurt,Non Fat Raspberry Yogurt,70.344028
22,Nonfat Icelandic Style Strawberry Yogurt,Vanilla Skyr Nonfat Yogurt,64.780041
25,Vanilla Skyr Nonfat Yogurt,Icelandic Style Skyr Blueberry Non-fat Yogurt,61.222024
27,Vanilla Skyr Nonfat Yogurt,Non Fat Raspberry Yogurt,55.151722
10,Total 2% Lowfat Greek Strained Yogurt With Blu...,Total 2% Lowfat Greek Strained Yogurt with Peach,48.870721
7,Total 2% Lowfat Greek Strained Yogurt With Blu...,Total 2% with Strawberry Lowfat Greek Strained...,48.343394
